### 16.2.2 GLMモデルの比較

### データを読み込む

In [ ]:
import polars as pl

housing = pl.read_csv("../data/housing_renamed.csv")
housing.head()

### 各モデルの学習

In [ ]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
def deviance_table(*models):
    return pl.DataFrame({
        "df_residuals": [mod.df_resid for mod in models],
        "resid_stddev": [mod.deviance for mod in models],
        "df": [mod.df_model for mod in models],
        "deviance": [mod.deviance for mod in models]
    })


f1 = "value_per_sq_ft ~ units + sq_ft + boro"
f2 = "value_per_sq_ft ~ units * sq_ft + boro"
f3 = "value_per_sq_ft ~ units + sq_ft * boro + type"
f4 = "value_per_sq_ft ~ units + sq_ft * boro + sq_ft * type"
f5 = "value_per_sq_ft ~ boro + type"

glm1 = smf.glm(f1, data = housing).fit()
glm2 = smf.glm(f2, data = housing).fit()
glm3 = smf.glm(f3, data = housing).fit()
glm4 = smf.glm(f4, data = housing).fit()
glm5 = smf.glm(f5, data = housing).fit()

glm_anova = deviance_table(glm1, glm2, glm3, glm4, glm5)
glm_anova

### ロジスティック回帰におけるモデル比較

In [ ]:
# 二値化
housing = (
    housing.with_columns([
        housing["value_per_sq_ft"].map_elements(lambda x: 1 if x >= 150 else 0).alias("high")
    ])
)
display(housing["high"].value_counts())

f1 = "high ~ units + sq_ft + boro"
f2 = "high ~ units * sq_ft + boro"
f3 = "high ~ units + sq_ft * boro + type"
f4 = "high ~ units + sq_ft * boro + sq_ft * type"
f5 = "high ~ boro + type"

logistic = statsmodels.genmod.families.family.Binomial(
    link = statsmodels.genmod.families.links.Logit()
)

glm1 = smf.glm(f1, data = housing, family = logistic).fit()
glm2 = smf.glm(f2, data = housing, family = logistic).fit()
glm3 = smf.glm(f3, data = housing, family = logistic).fit()
glm4 = smf.glm(f4, data = housing, family = logistic).fit()
glm5 = smf.glm(f5, data = housing, family = logistic).fit()

display(deviance_table(glm1, glm2, glm3, glm4, glm5))

In [ ]:
model_names = ["house1", "house2", "house3", "house4", "house5"]
mods = [glm1, glm2, glm3, glm4, glm5]

abic_glm = pl.DataFrame({
    "model": model_names,
    "aic": [mod.aic for mod in mods],
    "bic": [mod.bic for mod in mods]
})

abic_glm.sort(by = ["aic", "bic"])